In [4]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 11 11:02:03 2021

@author: Christian
"""

from mip import Model, xsum, maximize, BINARY

from Co2_price import df_data
from electricity_demand import electricity_demand
from Electricity_Price0 import Electricity_price_pivot0
from Electricity_Price1 import Electricity_price_pivot1
from gas_price import gas_p
from heat_demand import heat_demand_pivot
from heat_price import heat_p


elec_demand = electricity_demand
elec_price_wholesale = Electricity_price_pivot0
elec_price_household_32_05 = Electricity_price_pivot1
co2_price = df_data
gas_price = gas_p
heat_demand_1 = heat_demand_pivot
heat_price_1 = heat_p


el_price = elec_price_wholesale # Electricity price
del_t = .25 # duration of time step
el_demand = electricity_demand # Electicity demand


gas_p = gas_price # Gas price for different plants
em_fc = 0.2 # emission factor
co2_p = co2_price # CO2 Price
capacity_el = [5, 6, 7] # maximum capacity of electricity for the power plants
capacity_ht = [5, 6, 7] # maximum capacity of heat for the power plants
heat_demand = heat_demand_1 # heat demand
heat_price = heat_price_1 # heat price
heat_ratio = 1.21 # heat ratio


eff_plants = [0.33, 0.4, 0.45] # the efficiency of the power plants


T = range(len(el_price))
I = range(len(gas_p))

m = Model("Maximizing profit", sense=maximize)
 

#variable
y_t = [[m.add_var(lb = 0) for i in I] for t in T] # Electricity generation
el_sold = [m.add_var(lb = 0) for t in T] # electricity sold 
x_t = [[m.add_var(lb = 0) for i in I] for t in T] # Fuel consumption
z_t = [[m.add_var(lb = 0) for i in I] for t in T] # Heat generation
# objective function

# Revenue = Electricity price * demand/generation
# Cost = Fuel price = gas price + emission_factor * co2 price

# Max Proft = Revenue - Cost



In [5]:
elec_demand.head()

,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,63.975,54.150,49.075,48.625,56.700,65.400,75.400,103.625,168.375,225.850,...,172.050,185.200,199.950,204.275,182.175,126.875,98.850,86.85,77.250,72.800
2020-01-02,63.975,54.150,49.075,48.625,56.700,65.400,75.400,103.625,168.375,225.850,...,172.050,185.200,199.950,204.275,182.175,126.875,98.850,86.85,77.250,72.800
2020-01-03,63.975,54.150,49.075,48.625,56.700,65.400,75.400,103.625,168.375,225.850,...,172.050,185.200,199.950,204.275,182.175,126.875,98.850,86.85,77.250,72.800
2020-01-04,71.075,75.450,66.275,63.625,65.350,69.275,76.275,105.325,158.375,190.375,...,140.875,101.475,85.025,87.975,96.950,96.625,95.275,87.45,79.050,70.700
2020-01-05,60.500,54.525,47.825,42.900,41.875,43.100,48.150,51.525,49.475,52.850,...,76.575,68.025,62.125,73.150,90.750,93.550,91.550,80.85,71.975,63.825


In [6]:
elec_price_wholesale.head()

intHour,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
dtDate,,,,,,,,,,,,,,,,,,,,,
2020-01-01,3.485,4.188,3.860,3.655,3.232,3.085,3.014,3.017,3.000,3.065,...,3.004,3.075,3.211,3.598,4.040,4.405,4.315,4.345,4.068,4.027
2020-01-02,2.499,3.540,3.198,3.050,2.879,2.842,2.875,3.416,4.207,4.489,...,4.490,4.406,4.484,4.440,4.605,4.672,4.526,3.932,3.406,3.222
2020-01-03,2.204,2.147,1.304,0.153,0.014,0.085,0.992,2.448,2.668,2.881,...,3.790,3.811,3.791,3.844,4.047,4.135,3.337,2.889,2.770,2.570
2020-01-04,3.795,2.290,1.595,1.663,0.645,0.383,0.012,0.007,1.907,1.749,...,1.374,1.742,2.038,2.508,2.888,3.202,3.535,2.998,3.446,3.975
2020-01-05,3.049,3.310,3.228,3.118,3.010,2.996,2.988,3.038,3.115,3.209,...,3.537,3.444,3.610,4.059,4.468,4.616,4.462,3.950,3.576,3.649


In [7]:
elec_price_household_32_05.head()

intHour,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
dtDate,,,,,,,,,,,,,,,,,,,,,
2020-01-01,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,...,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05
2020-01-02,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,...,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05
2020-01-03,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,...,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05
2020-01-04,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,...,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05
2020-01-05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,...,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05,32.05


In [8]:
co2_price.head()

,mean_CO2_tax
Datum,
2020-01-01,17.43
2020-01-02,17.43
2020-01-03,17.43
2020-01-04,17.43
2020-01-05,19.93


In [9]:
gas_price.head()

,Preis
Datum,
2020-01-01,12.161
2020-01-02,9.775
2020-01-03,10.100
2020-01-04,7.987
2020-01-05,6.164


In [12]:
heat_demand.head()

hour,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
date,,,,,,,,,,,,,,,,,,,,,
2012-01-01,47077000.0,59491000.0,57734000.0,62913000.0,77677000.0,105238000.0,114154000.0,107996000.0,106766000.0,101548000.0,...,93100000.0,95736000.0,100419000.0,105237000.0,107291000.0,106845000.0,104586000.0,94116000.0,75726000.0,61090000.0
2012-01-02,52144000.0,49741000.0,50926000.0,56776000.0,68749000.0,86988000.0,100699000.0,99627000.0,99663000.0,92958000.0,...,82832000.0,85758000.0,89926000.0,93025000.0,96041000.0,95494000.0,91746000.0,82862000.0,64275000.0,51682000.0
2012-01-03,54257000.0,54816000.0,56107000.0,62414000.0,75600000.0,96777000.0,112554000.0,111367000.0,111580000.0,103283000.0,...,92399000.0,95435000.0,99348000.0,102940000.0,104696000.0,104469000.0,100877000.0,91066000.0,70257000.0,56659000.0
2012-01-04,59315000.0,57095000.0,58455000.0,65057000.0,78889000.0,101300000.0,118014000.0,116697000.0,116661000.0,107928000.0,...,96365000.0,99506000.0,103556000.0,107369000.0,109208000.0,109043000.0,105306000.0,95095000.0,73242000.0,59015000.0
2012-01-05,68047000.0,62181000.0,63678000.0,70265000.0,84014000.0,108208000.0,125904000.0,124340000.0,124283000.0,116018000.0,...,104023000.0,107197000.0,111498000.0,115099000.0,116679000.0,116125000.0,112165000.0,101370000.0,78775000.0,63980000.0


In [13]:
heat_price.head()

hour,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
date,,,,,,,,,,,,,,,,,,,,,
2012-01-01,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,...,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5
2012-01-02,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,...,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5
2012-01-03,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,...,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5
2012-01-04,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,...,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5
2012-01-05,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,...,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5


In [2]:
m.objective = xsum((el_price[t] * del_t * el_sold[t]) - ( x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t] * heat_price[t]) for t in T)

# constraints

for t in T:
    m += xsum(y_t[t][i] for i in I) == el_demand[t]
    
    for i in I:
        m += y_t[t][i] <= capacity_el[i] # electricity generation <= maximum capacity of electricity of the plant

        m += z_t[t][i] <= capacity_ht[i] # heat generation <= maximum capacity of heat of the plant 
        
        m += x_t[t][i] == y_t[t][i]/eff_plants[i] # fuel consumption = Electricity generation / efficiency of the plants 

        m += heat_demand[t] <= z_t[t][i] # heat demand >= Heat generation  

        m += y_t[t][i] == heat_ratio * z_t[t][i] # electricity generation >= heat ratio * heat generation

        m += y_t[t][i] == el_sold[t] + el_demand[t]



status = m.optimize()
obj= m.objective_value

41.709999999999994

In [3]:
status

<OptimizationStatus.OPTIMAL: 0>